In [ ]:
!pip install transformers[torch] datasets pandas rouge-score pyrogue

In [2]:
# from transformers import BertLMHeadModel, BertTokenizerFast
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda"
model_id = "EleutherAI/gpt-neo-2.7B"
model = AutoModelForCausalLM.from_pretrained(model_id, is_decoder=True, output_hidden_states=True).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [3]:
from datasets import load_dataset
import pandas as pd
# test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")

# test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
# encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")

0      For some time after its birth, it grows by gai...
1      When the bud on its back starts swelling, a sw...
2      After a rainy day, the flower on its back smel...
3      The fire on the tip of its tail is a measure o...
4      In the rocky mountains where Charmeleon live, ...
                             ...                        
796    One of the mysterious life-forms known as Ultr...
797    One kind of Ultra Beast. Witnesses have seen i...
798    This Ultra Beast came from the Ultra Wormhole....
799    A dangerous Ultra Beast, it appears to be eati...
800                                                    -
Name: description, Length: 801, dtype: object

# Perplexity

In [40]:
import torch
from tqdm import tqdm

test_sent = "Hello, nice to meet you."

def perplexity(sent):
    encodings = tokenizer(sent, return_tensors="pt")

    max_length = model.config.max_position_embeddings
    stride = 512
    seq_len = encodings.input_ids.size(1)

    nlls = []
    prev_end_loc = 0
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids, labels=target_ids)
            # https://stackoverflow.com/a/64796860/15557377
            action_logits = outputs.logits
            probs = torch.softmax(outputs.logits, dim = -1)
            sumProb = 0
            print(len(probs))
            for tensor in probs[0]:
                prob = torch.max(tensor)
                print(prob)
                sumProb += prob
            
            sent
            
            # Sum of all actions will equal the length of input as all distributions for each word prediction sum to 1
            # == print(len(torch.softmax(outputs.logits, dim = -1)[0])))
            # print(torch.softmax(outputs.logits, dim = -1))
            
            
            
            # probs = torch.nn.functional.softmax(last_hidden_state[mask_index])

            # loss is calculated using CrossEntropyLoss which averages over input tokens.
            # Multiply it with trg_len to get the summation instead of average.
            # We will take average over all the tokens to get the true average
            # in the last step of this example.
            neg_log_likelihood = outputs.loss * trg_len

        nlls.append(neg_log_likelihood)

        prev_end_loc = end_loc
        if end_loc == seq_len:
            break

    ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
    return ppl.item()

In [41]:
perplexity(test_sent)

  0%|          | 0/1 [00:00<?, ?it/s]

OI
1
tensor(0.2614, device='cuda:0')
tensor(0.1647, device='cuda:0')
tensor(0.7440, device='cuda:0')
tensor(0.7174, device='cuda:0')
tensor(0.9575, device='cuda:0')
tensor(0.3428, device='cuda:0')
tensor(0.2337, device='cuda:0')


6.1920857429504395

# SLOR

In [10]:
descriptions[0]

0      For some time after its birth, it grows by gai...
1      When the bud on its back starts swelling, a sw...
2      After a rainy day, the flower on its back smel...
3      The fire on the tip of its tail is a measure o...
4      In the rocky mountains where Charmeleon live, ...
                             ...                        
796    One of the mysterious life-forms known as Ultr...
797    One kind of Ultra Beast. Witnesses have seen i...
798    This Ultra Beast came from the Ultra Wormhole....
799    A dangerous Ultra Beast, it appears to be eati...
800                                                    -
Name: description, Length: 801, dtype: object

In [61]:
# Changed from perplexity calculation: https://huggingface.co/docs/transformers/perplexity

def sentProb(sent): 
    encodings = tokenizer(sent, return_tensors="pt")
    max_length = model.config.max_position_embeddings
    stride = 512
    
    seq_len = encodings.input_ids.size(1)
    
    
    nlls = []
    prev_end_loc = 0
    sentProb = 0
    for begin_loc in tqdm(range(0, seq_len, stride)):
        end_loc = min(begin_loc + max_length, seq_len)
        trg_len = end_loc - prev_end_loc  # may be different from stride on last loop
        input_ids = encodings.input_ids[:, begin_loc:end_loc].to(device)
        target_ids = input_ids.clone()
        target_ids[:, :-trg_len] = -100

        with torch.no_grad():
            outputs = model(input_ids)
            # https://stackoverflow.com/a/64796860/15557377
            action_logits = outputs.logits
            probs = torch.softmax(outputs.logits, dim = -1)
            for tensor in probs[0]:
                prob = torch.max(tensor)
                sentProb += prob
            
    
            # Sum of all actions will equal the length of input as all distributions for each word prediction sum to 1
            # == print(len(torch.softmax(outputs.logits, dim = -1)[0])))
            # print(torch.softmax(outputs.logits, dim = -1))
    sentLen = len(sent)
    sentProb = sentProb / sentLen
    return sentProb
    
def slor(sent):
    sentP = sentProb(sent)
    sumWordProb = 1
    ids = tokenizer.encode(sent.lower())
    tokens = tokenizer.convert_ids_to_tokens(ids)
    for token in tokens:
        sumWordProb = sumWordProb * sentProb(token)
    return 1/len(tokens) * ( torch.log(sentP) - torch.log(sumWordProb) ) 

In [62]:
print(slor(descriptions[0]))

100%|██████████| 1/1 [00:00<00:00, 17.74it/s]


['for', 'Ġsome', 'Ġtime', 'Ġafter', 'Ġits', 'Ġbirth', ',', 'Ġit', 'Ġgrows', 'Ġby', 'Ġgaining', 'Ġnour', 'ishment', 'Ġfrom', 'Ġthe', 'Ġseed', 'Ġon', 'Ġits', 'Ġback', '.']


100%|██████████| 1/1 [00:00<00:00, 53.65it/s]

tensor(2.2507, device='cuda:0')


# Error rate

In [5]:
#     errors
# 1 - ------
#     tokens 

In [ ]:
!pip install --upgrade language_tool_python
!pip install nltk
nltk.download('punkt')


In [10]:
import language_tool_python
tool = language_tool_python.LanguageToolPublicAPI('en-US')  # use a remote server (automatically set up), language English


In [30]:
from nltk.tokenize import word_tokenize

def errorRate(sent):
    check = tool.check(sent)
    numTokens = len(word_tokenize(sent))
    numErrors = len(check)
    print(check)
    print(numTokens)
    print(numErrors)
    return 1 - float(numErrors) / float(numTokens)

errorRate('helo darknes my old frend')

[Match({'ruleId': 'UPPERCASE_SENTENCE_START', 'message': 'This sentence does not start with an uppercase letter.', 'replacements': ['Helo'], 'offsetInContext': 0, 'context': 'helo darknes my old frend', 'offset': 0, 'errorLength': 4, 'category': 'CASING', 'ruleIssueType': 'typographical', 'sentence': 'helo darknes my old frend'}), Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'message': 'Possible spelling mistake found.', 'replacements': ['darkness', 'darkens', 'darkies'], 'offsetInContext': 5, 'context': 'helo darknes my old frend', 'offset': 5, 'errorLength': 7, 'category': 'TYPOS', 'ruleIssueType': 'misspelling', 'sentence': 'helo darknes my old frend'}), Match({'ruleId': 'MORFOLOGIK_RULE_EN_US', 'message': 'Possible spelling mistake found.', 'replacements': ['friend', 'trend', 'Fred', 'freed', 'Freud', 'Friend', 'fend', 'fiend', 'frond', 'rend', 'fr end'], 'offsetInContext': 20, 'context': 'helo darknes my old frend', 'offset': 20, 'errorLength': 5, 'category': 'TYPOS', 'ruleIssueType'

0.4

# Rogue

### Convert text to ROGUE format and evaluate

In [ ]:
from pyrouge import Rouge155

r = Rouge155()
r.system_dir = 'path/to/system_summaries'
r.model_dir = 'path/to/model_summaries'
r.system_filename_pattern = 'some_name.(\d+).txt'
r.model_filename_pattern = 'some_name.[A-Z].#ID#.txt'

output = r.convert_and_evaluate()
print(output)
output_dict = r.output_to_dict(output)

### Convert summaries to rogue 

In [ ]:
Rouge155.convert_summaries_to_rouge_format(system_input_dir, system_output_dir)

In [ ]:
from rouge_score import rouge_scorer
from pyrouge import Rouge155

scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
scores = scorer.score('The quick brown fox jumps over the lazy dog',
                      'The quick brown dog jumps on the log.')

